In [ ]:
instance = 1

In [ ]:
# DERIVE THE EOMs SYMBOLICALLY ----------------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np

sym.init_printing()
from IPython.display import display #for pretty printing

# create symbolic variables

# system parameters
g = sym.symbols('g')
mb,ml1,ml2 = sym.symbols(['m_{body}','m_{leg1}','m_{leg2}']) # mass
lb,ll1,ll2 = sym.symbols(['l_{body}','l_{leg1}','l_{leg2}']) # length
Inb,Inl1,Inl2 = sym.symbols(['I_{body}','I_{leg1}','I_{leg2}']) # moment of intertia

# generalized coordinates
x,y,thb,thl,r = sym.symbols(['x','y','\\theta_{body}','\\theta_{leg}','r']) 
dx,dy,dthb,dthl,dr = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{body}','\dot{\\theta}_{leg}','\dot{r}']) 
ddx,ddy,ddthb,ddthl,ddr = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{body}','\ddot{\\theta}_{leg}','\ddot{r}']) 

q = sym.Matrix([[x],[y],[thb],[thl],[r]])
dq = sym.Matrix([[dx],[dy],[dthb],[dthl],[dr]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb],[ddthl],[ddr]])

# forces
# total joint action = actuator + rebound, but that will be dealt with elsewhere
F,tau,GRFx,GRFy = sym.symbols(['F','\\tau','G_x','G_y']) 

# STEP 1: position vectors ri = [x,y,theta] (world frame)
rb = sym.Matrix([[x],
                [y],
                [thb]])

rl1 = sym.Matrix([[x + 0.5*ll1*sym.sin(thl)],
                [y - 0.5*ll1*sym.cos(thl)],
                [thl]])

rl2 = sym.Matrix([[x + (0.5*ll1+r)*sym.sin(thl)],
                [y - (0.5*ll1+r)*sym.cos(thl)],
                [thl]])

pfoot = sym.Matrix([[x + (ll1+r)*sym.sin(thl)],
                [y - (ll1+r)*sym.cos(thl)]])

# the Jacobians
Jb = rb.jacobian(q)
Jl1 = rl1.jacobian(q)
Jl2 = rl2.jacobian(q)

# STEP 2: generate expressions for the system space velocities from the jacobians
vb = Jb*dq
vl1 = Jl1*dq
vl2 = Jl2*dq

vfoot = pfoot.jacobian(q)*dq

# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb,Inb]])
Ml1 = sym.Matrix([[ml1,ml1,Inl1]])
Ml2 = sym.Matrix([[ml2,ml2,Inl2]])

T = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml1*sym.matrix_multiply_elementwise(vl1,vl1) + 0.5*Ml2*sym.matrix_multiply_elementwise(vl2,vl2)
T = T[0]
V = mb*g*rb[1] + ml1*g*rl1[1] + ml2*g*rl2[1]


# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(len(q),1)
for i in range(len(q)):
    dT_ddq = sym.Matrix([sym.diff(T,dq[i])]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = sym.Matrix([T]).jacobian(q).transpose() # partial of T in q

# term 4
Lg4 = sym.Matrix([V]).jacobian(q).transpose() # partial of U in q

# STEP 5: generalized forces

# force vectors for each link
tau_b = sym.Matrix([[0],[0],[-tau]])
tau_l1 = sym.Matrix([[0],[0],[tau]])

F_l1 = sym.Matrix([[-F*sym.sin(thl)],[F*sym.cos(thl)],[0]])
F_l2 = sym.Matrix([[F*sym.sin(thl)],[-F*sym.cos(thl)],[0]])

GRF_l2 = sym.Matrix([[GRFx],[GRFy],[0.5*ll2*GRFx*sym.cos(thl)+0.5*ll2*GRFy*sym.sin(thl)]])

Q = sym.zeros(len(q),1)
for j in range(len(q)):
    Q[j] = tau_b.transpose()*Jb[:,j]+(tau_l1+F_l1).transpose()*Jl1[:,j]+(F_l2+GRF_l2).transpose()*Jl2[:,j]

# AND combine!
EOM = Lg1 - Lg3 + Lg4 - Q


In [ ]:
# Lambdify
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,ml1,ml2,lb,ll1,ll2,Inb,Inl1,Inl2,
            x,y,thb,thl,r,
            dx,dy,dthb,dthl,dr,
            ddx,ddy,ddthb,ddthl,ddr,
            F,tau,GRFx,GRFy]

lambEOM = {}

DOFs = ['x','y','thb','thl','r']

for dof_i, dof in enumerate(DOFs):
    lambEOM.update({dof: sym.lambdify(sym_list,EOM[dof_i],modules = [func_map])})
    
lamb_footy = sym.lambdify(sym_list,pfoot[1],modules = [func_map])
lamb_dxfoot = sym.lambdify(sym_list,vfoot[0],modules = [func_map])

In [ ]:
# rerun from here if you don't want to calculate the EOM's again
if 'm' in globals():
    del m # deletes the model
    
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

N = 150
m.N = RangeSet(N) 

P = 3
m.P = RangeSet(P)

links = [('body',1),('leg',1),('leg',2)]
m.L = Set(dimen=2, initialize = links)

joints = ['hip','knee']
m.J = Set(initialize = joints)

DOFs = ['x','y','thb','thl','r'] # generalized coordinates
m.DOF = Set(initialize = DOFs) 

WDOFs = ['X','Y',"THETA"] # absolute coordinates 
m.WDOF = Set(initialize = WDOFs) 

signs = ['ps','ng'] # sign set for positive and negative components
m.sgn = Set(initialize = signs)

# PARAMETERS-----------------------------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)

def get_m(n, lb, ln):
    if lb == 'body':
        return 0.9
    else: return 0.05
m.m = Param(m.L, initialize = get_m) # mass of links

def get_len(n, lb, ln):
    if lb == 'body':
        return 0.6
    else: return 0.3
m.len = Param(m.L, initialize = get_len) # length of links

def calculate_In(m, lb, ln): 
    l = (lb,ln)
    # yes, that does mean you have to rebuild the tuple inside the function. Yes, it is dumb.
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

mbody = sum(m.m[l] for l in links)
BW = mbody*m.g.value

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.q = Var(m.N, m.P, m.DOF) # position
m.dq = Var(m.N, m.P, m.DOF) # velocity
m.ddq = Var(m.N, m.P, m.DOF) # acceleration

m.GRF = Var(m.N, m.P, m.WDOF, m.sgn, bounds = (0.0,None)) # ground reaction forces

m.force_a = Var(m.N, m.J) # actuator 
m.force_r = Var(m.N, m.P, m.J, m.sgn, bounds = (0,1)) # rebound 

# bound variables
for n in range(1,N+1):
    for p in range(1,P+1):
        m.q[n,p,'y'].setlb(0.0)
        m.q[n,p,'thb'].setlb(-np.pi/2)
        m.q[n,p,'thb'].setub(np.pi/2)
        m.GRF[n,p,'Y','ps'].setub(3.0)
            
# TIME AND INTEGRATION ---------------------------------------------------------------------------------------------------

# variable timestep
m.hm = Param(initialize = 0.02, mutable = True) # master timestep
m.h = Var(m.N, bounds = (0.8,1.2))

# Integration constraints 
radau_mat = [[0.19681547722366, -0.06553542585020, 0.02377097434822],
 [0.39442431473909, 0.29207341166523, -0.04154875212600],
 [0.37640306270047, 0.51248582618842, 0.11111111111111]]

# aux variable to assist with scaling
m.acc = Var(m.N, m.P, m.DOF)
m.vel = Var(m.N, m.P, m.DOF)

# RADAU
def get_acc(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.acc[n,p,dof] == m.hm*m.h[n]*sum([radau_mat[p-1][pp-1]*m.ddq[n,pp,dof] for pp in range(1,P+1)])
m.get_acc = Constraint(m.N, m.P, m.DOF, rule = get_acc)

def get_vel(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.vel[n,p,dof] == m.hm*m.h[n]*sum([radau_mat[p-1][pp-1]*m.dq[n,pp,dof] for pp in range(1,P+1)])
m.get_vel = Constraint(m.N, m.P, m.DOF, rule = get_vel)

def Radau_p(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.q[n,p,dof] == m.q[n-1,P,dof] + m.vel[n,p,dof]
m.integrate_p3 = Constraint(m.N, m.P, m.DOF, rule = Radau_p)

def Radau_v(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.dq[n,p,dof] == m.dq[n-1,P,dof] + m.acc[n,p,dof]
m.integrate_v3 = Constraint(m.N, m.P, m.DOF, rule = Radau_v)

# BACKWARD EULER
# Integration constraints 
def BwEuler_p(m,n,dof): # for positions
    if n > 1:
        return m.q[n,P,dof] == m.q[n-1,P,dof] + m.hm*m.h[n]*m.dq[n,P,dof]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p1 = Constraint(m.N, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,dof): # for velocities
    if n > 1:
        return m.dq[n,P,dof] == m.dq[n-1,P,dof] + m.hm*m.h[n]*m.ddq[n,P,dof]
    else:
        return Constraint.Skip 
m.integrate_v1 = Constraint(m.N, m.DOF, rule = BwEuler_v)

# GROUND INTERACTIONS -------------------------------------------------------------------------------------------------------

ground_constraints = ['contact','friction','slip_ps','slip_ng'] 
m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties

# variables
m.footy = Var(m.N, m.P, bounds = (0.0,None)) # foot position
m.footdx = Var(m.N, m.P, m.sgn, bounds = (0.0,None)) # foot velocity
m.friction_cone = Var(m.N, m.P, m.sgn, bounds = (0.0,None))
m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))

m.mu_k = Param(initialize = 1.2, mutable = True)
m.mu_s = Param(initialize = 2.4, mutable = True)
m.a_k = Var(m.N, m.P, bounds = (0.0, m.mu_s.value))

# constraints: aux variables
def def_footy(m,n,p):
    if n == 1 and p < P:
        return Constraint.Skip
    else:
        F_in = 0
        tau_in = 0
        Gx_in = 0
        Gy_in = 0
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
        m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
        m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
        m.q[n,p,'x'],m.q[n,p,'y'],m.q[n,p,'thb'],m.q[n,p,'thl'],m.q[n,p,'r'],
        m.dq[n,p,'x'],m.dq[n,p,'y'],m.dq[n,p,'thb'],m.dq[n,p,'thl'],m.dq[n,p,'r'],
        m.ddq[n,p,'x'],m.ddq[n,p,'y'],m.ddq[n,p,'thb'],m.ddq[n,p,'thl'],m.ddq[n,p,'r'],
        F_in,tau_in,Gx_in,Gy_in]
        
        return m.footy[n,p] == lamb_footy(*var_list)
m.def_footy = Constraint(m.N, m.P, rule = def_footy)

def def_friction_cone(m,n,p):
    if n == 1:
        return Constraint.Skip
    return m.friction_cone[n,p,'ps'] - m.friction_cone[n,p,'ng'] == m.a_k[n,p] - m.mu_k
m.def_friction_cone = Constraint(m.N, m.P, rule = def_friction_cone)

def def_footdx(m,n,p):
    if n == 1 and p < P:
        return Constraint.Skip
    else:
        F_in = 0
        tau_in = 0
        Gx_in = 0
        Gy_in = 0
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
        m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
        m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
        m.q[n,p,'x'],m.q[n,p,'y'],m.q[n,p,'thb'],m.q[n,p,'thl'],m.q[n,p,'r'],
        m.dq[n,p,'x'],m.dq[n,p,'y'],m.dq[n,p,'thb'],m.dq[n,p,'thl'],m.dq[n,p,'r'],
        m.ddq[n,p,'x'],m.ddq[n,p,'y'],m.ddq[n,p,'thb'],m.ddq[n,p,'thl'],m.ddq[n,p,'r'],
        F_in,tau_in,Gx_in,Gy_in]
        
        return m.footdx[n,p,'ps']-m.footdx[n,p,'ng'] == lamb_dxfoot(*var_list)
m.def_footdx = Constraint(m.N, m.P, rule = def_footdx)

def def_Gx(m,n,p):
    if n == 1:
        return Constraint.Skip
    return m.GRF[n,p,'X','ps'] + m.GRF[n,p,'X','ng'] == m.a_k[n,p]*m.GRF[n,p,'Y','ps']
m.def_Gx = Constraint(m.N, m.P, rule = def_Gx)

# constraints: complementarity
m.ground_varA = Var(m.N, m.ground_constraints)
m.ground_varB = Var(m.N, m.ground_constraints)

def def_ground_varA(m,n,gc):
    if n == 1:
        return Constraint.Skip
    if gc == 'contact':
        if n < N:
            return m.ground_varA[n,gc] == sum(m.footy[n+1,p] for p in range(1,P+1))
        else: 
            return m.ground_varA[n,gc] == sum(m.footy[n,p] for p in range(1,P+1))
    if gc == 'friction':
        return m.ground_varA[n,gc] == sum(m.footdx[n,p,'ps']+m.footdx[n,p,'ng'] for p in range(1,P+1))
    if gc == 'slip_ps':
        return m.ground_varA[n,gc] == sum(m.footdx[n,p,'ps'] for p in range(1,P+1))
    if gc == 'slip_ng':
        return m.ground_varA[n,gc] == sum(m.footdx[n,p,'ng'] for p in range(1,P+1))
m.def_ground_varA = Constraint(m.N, m.ground_constraints, rule = def_ground_varA)

def def_ground_varB(m,n,gc):
    if n == 1:
        return Constraint.Skip
    if gc == 'contact':
        return m.ground_varB[n,gc] == sum(BW*m.GRF[n,p,'Y','ps'] for p in range(1,P+1))
    if gc == 'friction':
        return m.ground_varB[n,gc] == sum(BW*m.friction_cone[n,p,'ps']+BW*m.friction_cone[n,p,'ng'] for p in range(1,P+1))
    if gc == 'slip_ps':
        return m.ground_varB[n,gc] == sum(BW*m.GRF[n,p,'X','ps'] for p in range(1,P+1))
    if gc == 'slip_ng':
        return m.ground_varB[n,gc] == sum(BW*m.GRF[n,p,'X','ng'] for p in range(1,P+1))
m.def_ground_varB = Constraint(m.N, m.ground_constraints, rule = def_ground_varB)

# complementarity
def ground_comp(m,n,gc):
    if n == 1:
        return Constraint.Skip
    return m.ground_penalty[n,gc] == m.ground_varA[n,gc]*m.ground_varB[n,gc]
m.ground_comp = Constraint(m.N, m.ground_constraints, rule = ground_comp)

# HARD JOINT STOPS ----------------------------------------------------------------------------------------------------------

joint_constraints = ['up','lo'] # set of joint penalties
m.joint_constraints = Set(initialize = joint_constraints)

# variables
m.joint_penalty = Var(m.N, m.J, m.joint_constraints, bounds = (0.0,None))
m.joint_varA = Var(m.N, m.J, m.joint_constraints)
m.joint_varB = Var(m.N, m.J, m.joint_constraints)

m.jointp = Var(m.N, m.P, m.J)
m.jointv = Var(m.N, m.P, m.J)
m.power = Var(m.N, m.P, m.J)

for n in range(1,N+1):
    for p in range(1,P+1):
        m.jointp[n,p,'hip'].setlb(-np.pi/2)
        m.jointp[n,p,'hip'].setub(np.pi/2)
        m.jointp[n,p,'knee'].setlb(0)
        m.jointp[n,p,'knee'].setub(0.3)
        
def def_jointp(m,n,p,j):
    if n == 1 and p < P:
        return Constraint.Skip
    if j == 'hip':
        return m.jointp[n,p,j] == m.q[n,p,'thl'] - m.q[n,p,'thb']
    if j == 'knee':
        return m.jointp[n,p,j] == m.q[n,p,'r']
m.def_jointp = Constraint(m.N, m.P, m.J, rule = def_jointp)

def def_jointv(m,n,p,j):
    if n == 1 and p < P:
        return Constraint.Skip
    if j == 'hip':
        return m.jointv[n,p,j] == m.dq[n,p,'thl'] - m.dq[n,p,'thb']
    if j == 'knee':
        return m.jointv[n,p,j] == m.dq[n,p,'r']
m.def_jointv = Constraint(m.N, m.P, m.J, rule = def_jointv)

def def_power(m,n,p,j):
    if n == 1:
        return Constraint.Skip
    else:
        return m.power[n,p,j] == 9.81*m.force_a[n,j]*m.jointv[n,p,j]
m.def_power = Constraint(m.N, m.P, m.J, rule = def_power)

def def_joint_varA(m,n,j,jc):
    if n == 1:
        return Constraint.Skip
    if jc == 'up':
        if n < N:
            return m.joint_varA[n,j,jc] == sum(m.jointp[3,P,j].ub - m.jointp[n+1,p,j] for p in range(1,P+1))
        else:
            return m.joint_varA[n,j,jc] == sum(m.jointp[3,P,j].ub - m.jointp[n,p,j] for p in range(1,P+1))
    if jc == 'lo':
        if n < N:
            return m.joint_varA[n,j,jc] == sum(m.jointp[n+1,p,j] - m.jointp[3,P,j].lb for p in range(1,P+1))
        else:
            return m.joint_varA[n,j,jc] == sum(m.jointp[n,p,j] - m.jointp[3,P,j].lb for p in range(1,P+1))
m.def_joint_varA = Constraint(m.N, m.J, m.joint_constraints, rule = def_joint_varA)

# define aux variables
def def_joint_varB(m,n,j,jc):
    if n == 1:
        return Constraint.Skip
    if jc == 'up':
        return m.joint_varB[n,j,jc] == sum(BW*m.force_r[n,p,j,'ng'] for p in range(1,P+1))
    if jc == 'lo':
        return m.joint_varB[n,j,jc] == sum(BW*m.force_r[n,p,j,'ps'] for p in range(1,P+1))
m.def_joint_varB = Constraint(m.N, m.J, m.joint_constraints, rule = def_joint_varB)

# complementarity
def joint_comp(m,n,j,jc):
    if n == 1:
        return Constraint.Skip
    return m.joint_penalty[n,j,jc] == m.joint_varA[n,j,jc]*m.joint_varB[n,j,jc]
m.joint_comp = Constraint(m.N, m.J, m.joint_constraints, rule = joint_comp)

# stop ends of links from going through the floor
m.jointy = Var(m.N, m.P, m.J, bounds = (0.0,None))

# DYNAMICS ----------------------------------------------------------------------------------------------------------------
S = BW

def dynamics(m,n,p,dof):
    if n == 1:
        return Constraint.Skip
    else:
        F_in = m.force_a[n,'knee'] + m.force_r[n,p,'knee','ps'] - m.force_r[n,p,'knee','ng']
        tau_in = m.force_a[n,'hip'] + m.force_r[n,p,'hip','ps'] - m.force_r[n,p,'hip','ng']
        Gx_in = m.GRF[n,p,'X','ps'] - m.GRF[n,p,'X','ng']
        Gy_in = m.GRF[n,p,'Y','ps']
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
        m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
        m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
        m.q[n,p,'x'],m.q[n,p,'y'],m.q[n,p,'thb'],m.q[n,p,'thl'],m.q[n,p,'r'],
        m.dq[n,p,'x'],m.dq[n,p,'y'],m.dq[n,p,'thb'],m.dq[n,p,'thl'],m.dq[n,p,'r'],
        m.ddq[n,p,'x'],m.ddq[n,p,'y'],m.ddq[n,p,'thb'],m.ddq[n,p,'thl'],m.ddq[n,p,'r'],
        S*F_in,S*tau_in,S*Gx_in,S*Gy_in]
    
    return lambEOM[dof](*var_list) == 0
m.dynamics = Constraint(m.N, m.P, m.DOF, rule = dynamics)

# OBJECTIVE --------------------------------------------------------------------------------------------------------------
def MinPenalty(m):
    penalty_sum = 0
    for n in range(1,N+1):
        for gc in ground_constraints:
            penalty_sum += m.ground_penalty[n,gc]
        for j in joints:
            for jc in joint_constraints:
                penalty_sum += m.joint_penalty[n,j,jc]
    return penalty_sum
m.MinPenalty = Objective(rule = MinPenalty)

In [ ]:
def init_opt(): # initializes solver
    opt = SolverFactory('ipopt',executable = 'C:/cygwin64/home/Stacey/CoinIpopt/build/bin/ipopt.exe')
    opt.options["linear_solver"] = 'ma97'
    opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
    opt.options["max_iter"] = 30000 # maximum number of iterations
    opt.options["max_cpu_time"] = 3600 # maximum cpu time in seconds
    opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
    opt.options["OF_acceptable_obj_change_tol"] = 1e-4
    opt.options["OF_ma86_scaling"] = 'none'
    
    return opt

In [ ]:
def euler_mode(m):
    m.integrate_p1.activate()
    m.integrate_v1.activate()
    
    # deactivate constraints
    m.get_acc.deactivate()
    m.get_vel.deactivate()
    m.integrate_p3.deactivate()
    m.integrate_v3.deactivate()

    for n in range(2,N+1):
        for p in range(1,P):
            m.def_footy[n,p].deactivate()
            m.def_footdx[n,p].deactivate()
            m.def_Gx[n,p].deactivate()
            m.def_friction_cone[n,p].deactivate()
            for j in joints:
                m.def_jointp[n,p,j].deactivate()
                m.def_jointv[n,p,j].deactivate()
                m.def_power[n,p,j].deactivate()
            for dof in DOFs:
                m.dynamics[n,p,dof].deactivate()

#     fix variables
    for n in range(2,N+1):
        for p in range(1,P):
            for dof in DOFs:
                m.q[n,p,dof].fix(0.0)
                m.dq[n,p,dof].fix(0.0)
                m.ddq[n,p,dof].fix(0.0)
            
            m.a_k[n,p].fix(0.0)
            for dof in WDOFs:
                for sgn in signs:
                    m.GRF[n,p,dof,sgn].fix(0.0)
                    m.footy[n,p].fix(0.0)
                    m.footdx[n,p,sgn].fix(0.0)
                    m.friction_cone[n,p,sgn].fix(0.0)
            for j in joints:
                for sgn in signs:
                    m.force_r[n,p,j,sgn].fix(0.0)
                    m.jointp[n,p,j].fix(0.0)
                    m.jointv[n,p,j].fix(0.0)
                    m.power[n,p,j].fix(0.0)                                  
    return m

def radau_mode(m):
    m.integrate_p1.deactivate()
    m.integrate_v1.deactivate()
    
    # deactivate constraints
    m.get_acc.activate()
    m.get_vel.activate()
    m.integrate_p3.activate()
    m.integrate_v3.activate()

    m.def_footy.activate()
    m.def_footdx.activate()
    m.def_Gx.activate()
    m.def_friction_cone.activate()
    m.def_jointp.activate()
    m.def_jointv.activate()
    m.def_power.activate()
    m.dynamics.activate()

#   unfix variables
    for n in range(2,N+1):
        for p in range(1,P):
            for dof in DOFs:
                m.q[n,p,dof].unfix()
                m.dq[n,p,dof].unfix()
                m.ddq[n,p,dof].unfix()
            
            m.a_k[n,p].unfix()
            for dof in WDOFs:
                for sgn in signs:
                    m.GRF[n,p,dof,sgn].unfix()
                    m.footy[n,p].unfix()
                    m.footdx[n,p,sgn].unfix()
                    m.friction_cone[n,p,sgn].unfix()
            for j in joints:
                for sgn in signs:
                    m.force_r[n,p,j,sgn].unfix()
                    m.jointp[n,p,j].unfix()
                    m.jointv[n,p,j].unfix()
                    m.power[n,p,j].unfix()                                 
    return m

def init_midpoints(m):
    for n in range(2,N+1):
        for p in range(1,P):
            for dof in DOFs:
                m.q[n,p,dof].value = m.q[n,P,dof].value
                m.dq[n,p,dof].value = m.dq[n,P,dof].value
                m.ddq[n,p,dof].value = m.ddq[n,P,dof].value
            
            m.a_k[n,p].value = m.a_k[n,P].value
            for dof in WDOFs:
                for sgn in signs:
                    m.GRF[n,p,dof,sgn].value = m.GRF[n,P,dof,sgn].value
                    m.footy[n,p].value = m.footy[n,P].value
                    m.footdx[n,p,sgn].value = m.footdx[n,P,sgn].value
                    m.friction_cone[n,p,sgn].value = m.friction_cone[n,P,sgn].value
            for j in joints:
                for sgn in signs:
                    m.force_r[n,p,j,sgn].value = m.force_r[n,P,j,sgn].value
                    m.jointp[n,p,j].value = m.jointp[n,P,j].value
                    m.jointv[n,p,j].value = m.jointv[n,P,j].value
                    m.power[n,p,j].value = m.power[n,P,j].value                                
    return m

In [ ]:
# 3 second sprint from rest
T = 3.0
m.hm = T/N

# initial condition
for dof in DOFs:
    m.dq[1,P,dof].fix(0)
    
m.q[1,P,'x'].fix(0)
m.q[1,P,'thb'].fix(0)
m.q[1,P,'thl'].fix(0)
m.q[1,P,'r'].fix(0.3)

# final condition
m.q[N,P,'x'].setlb(10)

# max height (so the overpowered model can't just leap the whole way :P)
for n in range(1,N+1):
    for p in range(1,P+1):
        m.q[n,p,'y'].setub(1)

In [ ]:
# STAGE 1 euler solve
m = euler_mode(m)
opt = init_opt()
results = opt.solve(m, tee = True)

In [ ]:
# STAGE 2 radau solve
m = radau_mode(m)
results = opt.solve(m, tee = True)

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1,dpi=100) #create axes
ax1.set_aspect('equal')

xmax = np.max([m.q[n,P,'x'].value for n in range(1,N+1)])

def plot_robot(i,P,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-1,xmax+1])
    ax.set_ylim([0,1.5])
    
    #plot body
    body_xb = m.q[i,P,'x'].value - 0.5*m.len[('body',1)]*cos(m.q[i,P,'thb'].value)
    body_yb = m.q[i,P,'y'].value - 0.5*m.len[('body',1)]*sin(m.q[i,P,'thb'].value)
    body_xf = m.q[i,P,'x'].value + 0.5*m.len[('body',1)]*cos(m.q[i,P,'thb'].value)
    body_yf = m.q[i,P,'y'].value + 0.5*m.len[('body',1)]*sin(m.q[i,P,'thb'].value)  
    ax.plot([body_xb,body_xf],[body_yb,body_yf],color='xkcd:black')
      
    #plot leg 1
    thA = m.q[i,P,'thl'].value
    leg1_xt = m.q[i,P,'x'].value
    leg1_yt = m.q[i,P,'y'].value
    leg1_xb = m.q[i,P,'x'].value + m.len[('leg',1)]*sin(thA)
    leg1_yb = m.q[i,P,'y'].value - m.len[('leg',1)]*cos(thA)
    ax.plot([leg1_xt,leg1_xb],[leg1_yt,leg1_yb],color='xkcd:red')
    
    #plot leg 2
    Lt = 0.5*m.len[('leg',1)] + m.q[i,P,'r'].value - 0.5*m.len[('leg',2)]
    Lb = 0.5*m.len[('leg',1)] + m.q[i,P,'r'].value + 0.5*m.len[('leg',2)]
    leg2_xt = m.q[i,P,'x'].value + Lt*sin(thA)
    leg2_yt = m.q[i,P,'y'].value - Lt*cos(thA)
    leg2_xb = m.q[i,P,'x'].value + Lb*sin(thA)
    leg2_yb = m.q[i,P,'y'].value - Lb*cos(thA)
    ax.plot([leg2_xt,leg2_xb],[leg2_yt,leg2_yb],color='xkcd:orange')
    
update = lambda i: plot_robot(i,P,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1,3),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
